In [1]:
from pycocotools.coco import COCO
import os
import json
os.chdir("../")
os.getcwd()

'/workspace'

In [2]:
coco_train = COCO("./data/COCO/Annotations/instances_train2014.json")
coco_train_cap = COCO("./data/COCO/Annotations/captions_train2014.json")
coco_val = COCO("./data/COCO/Annotations/instances_val2014.json")
coco_val_cap = COCO("./data/COCO/Annotations/captions_val2014.json")

loading annotations into memory...
Done (t=8.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.55s)
creating index...
index created!
loading annotations into memory...
Done (t=4.21s)
creating index...
index created!
loading annotations into memory...
Done (t=0.23s)
creating index...
index created!


In [3]:
plural_map = {'person': 'people', 'bus': 'buses', 'bench':'benches','couch': 'couches', 'sandwich': 'sandwiches', 'scissors':'scissorses', 'knife':'knives', 'toothbrush':'toothbrushes'}

def pluraliize(word):
    if word in plural_map:
        return plural_map[word]
    else:
        return word + 's'

In [4]:
def get_img_info(coco, coco_cap):
    outputs = {}
    for i in coco.getImgIds():
        img_info = {}
        img_info['file_name'] = coco.loadImgs(i)[0]['file_name']
        img_info['width'] = coco.loadImgs(i)[0]['width']
        img_info['height'] = coco.loadImgs(i)[0]['height']
        
        ann_ids = coco.getAnnIds(imgIds=i, catIds=[], iscrowd=False)
        anns = coco.loadAnns(ann_ids)
                
        img_info['objects_count'] = len(anns)
        if not anns:
            img_info.update({
                'max_obj_id': None,
                'max_obj_cat_id': None,
                'max_obj_super_cat': None,
                'max_obj_cat': None,
                'max_obj_area': None,
                'max_obj_midpoint': None,
                'max_obj_bbox': None,
                'max_obj_segment_points': None,
                'max_obj_area_portion': None,
                'captions': None,
            })
        else:
            sorted_anns = sorted(anns, key=lambda x: x['area'], reverse=True)
            max_ann = sorted_anns[0]
            
            img_info['max_obj_id'] = max_ann['id']
            img_info['max_obj_cat_id'] = max_ann['category_id']
            img_info['max_obj_super_cat'] = [coco.cats[max_ann['category_id']]['supercategory'], pluraliize(coco.cats[max_ann['category_id']]['supercategory'])]
            img_info['max_obj_cat'] = coco.cats[max_ann['category_id']]['name'], pluraliize(coco.cats[max_ann['category_id']]['name'])
            img_info['max_obj_area'] = max_ann['area']
            img_info['max_obj_midpoint'] = [max_ann['bbox'][0] + max_ann['bbox'][2] / 2, max_ann['bbox'][1] + max_ann['bbox'][3] / 2]
            img_info['max_obj_bbox'] = max_ann['bbox']
            img_info['max_obj_segment_points'] = max_ann['segmentation']
            img_info['max_obj_area_portion'] = max_ann['area'] / (coco.loadImgs(i)[0]['height'] * coco.loadImgs(i)[0]['width'])
            img_info["captions"] = [coco_cap.anns[annId]["caption"] for annId in coco_cap.getAnnIds(imgIds=i)]
        
        outputs[i] = img_info
        
    return outputs

In [5]:
output_val = get_img_info(coco_val, coco_val_cap)
output_train = get_img_info(coco_train, coco_train_cap)
output = {**output_train, **output_val}
with open('./data/COCO/Annotations/coco_img_info.json', 'w') as f:
    json.dump(output, f)